In [3]:
# !pip install selenium beautifulsoup4 pandas

In [2]:
# from selenium import webdriver
# from selenium.webdriver.chrome.options import Options
# from bs4 import BeautifulSoup
# import pandas as pd
# import time

# # Setup Selenium
# options = Options()
# options.add_argument("--headless")  # Run in headless mode
# driver = webdriver.Chrome(options=options)

# # Base URL
# base_url = "https://www.sbtjapan.com/used-cars/search?is_new_available=1&page={}"

# # Data container
# cars = []

# # Scrape first 10 pages (can be increased)
# for page in range(1, 11):
#     driver.get(base_url.format(page))
#     time.sleep(3)  # Wait for page to load

#     soup = BeautifulSoup(driver.page_source, 'html.parser')
#     listings = soup.find_all("div", class_="vehicle-card")

#     for car in listings:
#         try:
#             car_brand = car.find("div", class_="maker-name").text.strip()
#             car_model = car.find("div", class_="model-name").text.strip()
#             specs = car.find("ul", class_="vehicle-feature-list").text.strip()
#             price_text = car.find("span", class_="vehicle-price").text.strip().replace(",", "").replace("Ksh", "").strip()
#             price = float(price_text)

#             mileage_km = None
#             engine_size_cc = None
#             transmission = None
#             fuel_type = None
#             year = None

#             # Parse spec items
#             for spec in car.find_all("li"):
#                 text = spec.text.lower()
#                 if "km" in text:
#                     mileage_km = float(text.replace("km", "").replace(",", "").strip())
#                 elif "cc" in text:
#                     engine_size_cc = float(text.replace("cc", "").replace(",", "").strip())
#                 elif "transmission" in text:
#                     transmission = text.split(":")[-1].strip()
#                 elif "fuel" in text:
#                     fuel_type = text.split(":")[-1].strip()
#                 elif "year" in text:
#                     year = int(text.split(":")[-1].strip())

#             cars.append({
#                 "car_brand": car_brand,
#                 "car_model": car_model,
#                 "mileage_km": mileage_km,
#                 "engine_size_cc": engine_size_cc,
#                 "transmission": transmission,
#                 "fuel_type": fuel_type,
#                 "year": year,
#                 "price(Ksh)": price
#             })

#         except Exception as e:
#             print(f"Error: {e}")
#             continue

#     print(f"Page {page} scraped.")

# # Close the driver
# driver.quit()

# # Convert to DataFrame
# df = pd.DataFrame(cars)
# df.to_csv("sbt_used_cars.csv", index=False)
# print("Scraping complete. Saved to sbt_used_cars.csv")


Page 1 scraped.
Page 2 scraped.
Page 3 scraped.
Page 4 scraped.
Page 5 scraped.
Page 6 scraped.
Page 7 scraped.
Page 8 scraped.
Page 9 scraped.
Page 10 scraped.
Scraping complete. Saved to sbt_used_cars.csv


In [5]:
import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Setup
options = Options()
options.add_argument("--headless")
options.add_argument("--disable-gpu")

driver = webdriver.Chrome(service=Service(), options=options)
wait = WebDriverWait(driver, 10)

base_url = "https://www.sbtjapan.com/used-cars/search?is_new_available=1&page={}"
results = []

# Pages to scrape
for page in range(1, 4):  # Scrape first 3 pages for testing
    driver.get(base_url.format(page))

    try:
        wait.until(EC.presence_of_element_located((By.CLASS_NAME, "vehicle-card")))
    except:
        print(f"Listings not found on page {page}")
        continue

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    listings = soup.find_all("div", class_="vehicle-card")

    for listing in listings:
        try:
            brand = listing.find("div", class_="maker-name")
            model = listing.find("div", class_="model-name")
            price = listing.find("span", class_="vehicle-price")
            specs = listing.find_all("li")

            data = {
                "car_brand": brand.text.strip() if brand else None,
                "car_model": model.text.strip() if model else None,
                "price(Ksh)": price.text.replace(",", "").replace("Ksh", "").strip() if price else None,
                "mileage_km": None,
                "engine_size_cc": None,
                "year": None,
                "fuel_type": None,
                "transmission": None
            }

            for spec in specs:
                text = spec.get_text().lower()
                if "km" in text and "mileage" in text:
                    data['mileage_km'] = float(text.split(":")[-1].replace("km", "").replace(",", "").strip())
                elif "cc" in text:
                    data['engine_size_cc'] = float(text.replace("cc", "").replace(",", "").strip())
                elif "year" in text:
                    data['year'] = int(text[-4:])
                elif "fuel" in text:
                    data['fuel_type'] = text.split(":")[-1].strip()
                elif "transmission" in text:
                    data['transmission'] = text.split(":")[-1].strip()

            results.append(data)

        except Exception as e:
            print("Error in a car listing:", e)
            continue

# Save to CSV
df = pd.DataFrame(results)
df.to_csv("sbtjapan_cars_fixed.csv", index=False)
print("✅ Data saved to sbtjapan_cars_fixed.csv")

driver.quit()


Listings not found on page 1
Listings not found on page 2
Listings not found on page 3
✅ Data saved to sbtjapan_cars_fixed.csv
